In [6]:
!pip install spacy nltk
!spacy download en_core_web_sm
!pip uninstall tensorflow-gpu -y
!pip uninstall tensorflow -y
!pip install --upgrade tensorflow

  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/confection/__init__.py", line 38, in <module>
    from pydantic.v1 import BaseModel, Extra, ValidationError, create_model
  File "/usr/local/lib/python3.11/dist-packages/pydantic/__init__.py", line 13, in <module>
    from . import dataclasses
  File "/usr/local/lib/python3.11/dist-packages/pydantic/dataclasses.py", line 11, in <module>
    from ._internal imp

In [7]:
import spacy
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Embedding
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf
from nltk.util import ngrams
import time
import csv

## utilize TPUs
the code below will create a TPU strategy if there are any TPUs availabe

In [8]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver('') 
    print(tpu)
    print('Running on TPU ', tpu.master())
except Exception as e:
    print(e)
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

tf.config.experimental.list_physical_devices()

Please provide a TPU Name to connect to.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## utilize multiple GPUs
the code below will create a mirror stategy which will allow multiple CPUs to work together.

In [9]:
mirrored_strategy = tf.distribute.MirroredStrategy()
#tf.config.set_soft_device_placement(True)
tf.test.is_gpu_available()
print('DEVICES AVAILABLE: {}'.format(mirrored_strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
DEVICES AVAILABLE: 1


## loading the small english core model from spacy
in the code bellow we are loading the small english web core from spacy while disabling parser, tagger, ner, since we are not gonna use these features in our model, disabling them will make the text processing faster.

In [11]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
nlp.add_pipe('sentencizer')
nlp.max_length = 6_000_000_000

spacy.prefer_gpu()

False

In [12]:
word_index_file = "word_index.json"
stories_folder = "stories"
stories_files = os.listdir(stories_folder)

model_progress = 'result/model_progress.json'
models_path = 'result/working/models'
history_folder = 'result/working/history'

FileNotFoundError: [Errno 2] No such file or directory: 'stories'